In [2]:
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from bpe import BayesPE 
from llm_model import LLM
import evaluation
import constants

In [3]:
# Define task instructions
instructions = [
    "Is the following Youtube comment spam?",
    "Determine whether the given Youtube comment is spam.",
    "Classify the following Youtube comment as spam or not.",
    "Check if the Youtube comment below is considered spam.",
    "Decide if the given Youtube comment is spam.",
    "Is this Youtube comment a form of spam?",
    "Evaluate whether this Youtube comment qualifies as spam.",
    "Identify if the following comment on Youtube is spam.",
    "Judge whether the Youtube comment should be flagged as spam."
]

In [4]:
# Load youtube comments train and test data
df_train = pd.read_csv('youtube.csv')
n_train = 1100  
n_in_context = 5 
n_total_in_context = len(instructions) * n_in_context  
n_val=100
df_train_actual = df_train.iloc[:n_train] 
df_in_context_base = df_train.iloc[n_train:n_train + n_total_in_context]
df_val = df_train.iloc[n_train + n_total_in_context:n_train+n_total_in_context+n_val]
df_test_actual = df_train.iloc[n_train+n_total_in_context+n_val:]  
gt_labels_train = df_train_actual.iloc[:, 4].values.astype(int) 
samples_train = df_train_actual.iloc[:, 3].values 
gt_labels_val = df_val.iloc[:, 4].values.astype(int) 
samples_val = df_val.iloc[:, 3].values 
gt_labels_test = df_test_actual.iloc[:, 4].values.astype(int)
samples_test = df_test_actual.iloc[:, 3].values 

# **Prepare Unique In-Context Examples Per Instruction**
for i in range(len(instructions)):  
    start_idx = i * n_in_context
    end_idx = (i + 1) * n_in_context
    df_in_context = df_in_context_base.iloc[start_idx:end_idx]

    samples_in_context_i = df_in_context.iloc[:, 3].values
    gt_labels_in_context_i = df_in_context.iloc[:, 4].values.astype(int)

    if i == 0:
        samples_in_context = np.expand_dims(samples_in_context_i, axis=1)
        gt_labels_in_context = np.expand_dims(gt_labels_in_context_i, axis=1)
    else:
        samples_in_context = np.concatenate((samples_in_context, np.expand_dims(samples_in_context_i, axis=1)), axis=1)
        gt_labels_in_context = np.concatenate((gt_labels_in_context, np.expand_dims(gt_labels_in_context_i, axis=1)), axis=1)


In [5]:
# Define prompt Formatting Class for youTube comment spam detection and initialize an LLM-based classifier
class PromptFormatting(object):
    def __init__(self):
        self.INSTRUCTION = 'Is the following Youtube comment spam?'
        self.CLASSES = ['not spam', 'spam']
        self.CLASSES_FOR_MATCHING = [self.CLASSES, ['ham', 'spam'], ['0', '1']]
        self.CLASSES_TEXT = '''1. {}\n2. {}'''.format(self.CLASSES[0], self.CLASSES[1])

    def format_instruction(self, instruction):
        return '''{}\n{}\n'''.format(instruction, self.CLASSES_TEXT)

    def format_content(self, content):
        return '''comment: {}\nthe comment is '''.format(content)

prompt_formatting = PromptFormatting()



# Initialize BayesPE (Teacher Model)
bayespe_classifier = BayesPE(
    model_name="mistralai/Mistral-7B-Instruct-v0.3", 
    prompt_formatting=prompt_formatting,
    instructions=instructions, 
    few_shot_texts_sets=samples_in_context, 
    few_shot_labels_sets=gt_labels_in_context, 
    use_reduced_precision=True
)

# Print example prompt
bayespe_classifier.print_prompt_example()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

EXAMPLE 1:
Is the following Youtube comment spam?
1. not spam
2. spam

comment: Check out my music niggas﻿
the comment is spam

EXAMPLE 2:
Is the following Youtube comment spam?
1. not spam
2. spam

comment: Check out this video on YouTube:﻿
the comment is spam

EXAMPLE 3:
Is the following Youtube comment spam?
1. not spam
2. spam

comment: Check out this funny video &quot;Cereal Box Knocks out Baby&quot; on my channel.﻿
the comment is spam

EXAMPLE 4:
Is the following Youtube comment spam?
1. not spam
2. spam

comment: Subscribe to me if u think &quot;swag&quot; is fucking stupid﻿
the comment is spam

EXAMPLE 5:
Is the following Youtube comment spam?
1. not spam
2. spam

comment: Tuto to subscribe to my channel because you should sign up for 17 l please thank you I&#39;d do anything for you to sign up a lot of good video I usually do!﻿
the comment is spam

EXAMPLE 6:
Is the following Youtube comment spam?
1. not spam
2. spam

comment: <SAMPLE_IN>
the comment is <LABEL_OUT>


In [6]:
# Optimize prompt weights
bayespe_classifier.optimise_weights(samples_val, gt_labels_val)

inference for promt 1 out of 9
Almost 1 billion﻿
Aslamu Lykum... From Pakistan﻿
Eminem is idol for very people in España and Mexico or Latinoamerica﻿
Help me get 50 subs please ﻿
i love song :)﻿
Alright ladies, if you like this song, then check out John Rage.  He&#39;s a smoking hot rapper coming into the game.  He&#39;s not better than Eminem lyrically, but he&#39;s hotter. Hear some of his songs on my channel.﻿
The perfect example of abuse from husbands and the thing is I&#39;m a feminist so I definitely agree with this song and well...if I see this someone&#39;s going to die! Just sayin.﻿
The boyfriend was Charlie from the TV show LOST ﻿
<a href="https://www.facebook.com/groups/100877300245414/">https://www.facebook.com/groups/100877300245414/</a>﻿
Take a look at this video on YouTube:﻿
Check out our Channel for nice Beats!!﻿
Rihanna and Eminem together are unstoppable.﻿
Check out this playlist on YouTube:﻿
This song is about Rape and Cheating     <br /><br /><br /><br /><br /><br /

array([0.03955476, 0.17900996, 0.01519829, 0.01813601, 0.00779092,
       0.18382725, 0.0519125 , 0.075511  , 0.42905927], dtype=float32)

In [ ]:
# Get prompt weights and prompt wise probabilities on train data
_,probs,weights = bayespe_classifier.forward(samples_train, n_forward_passes=9)

In [ ]:
torch.save(probs,'youtube_probs.pt')
torch.save(weights,'youtube_prompt_weights.pt')

In [ ]:
# Evaluate BayesPE performance on youtube comments test data
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
teacher_probs,_,_ = bayespe_classifier.forward(samples_test, n_forward_passes=9)
end.record()

# Wait for the events to be recorded
torch.cuda.synchronize()

# Report in seconds
elapsed_time_ms = start.elapsed_time(end)  # in milliseconds
elapsed_time_sec = elapsed_time_ms / 1000  # convert to seconds

print(f"Inference time: {elapsed_time_sec:.4f} seconds")
print(teacher_probs[:10, :])
f1_score = evaluation.compute_metric(gt_labels_test, teacher_probs, metric='f1')
ece = evaluation.compute_metric(gt_labels_test, teacher_probs, metric='ece')
print('Teacher f1-score: {}, Teacher ECE: {}'.format(f1_score, ece))